## Цель работы

Ознакомиться с методами одномерного поиска, используемыми в многмерных методах минимизации функции n переменных. Сравнить различные алгоритмы по эффективности на тестовых примерах.

## Задание

* Реализовать методы дихотомии, золотого сечения, исследовать их сходимость и провести сравнение по числу вычислений функций для достижения заданной точности $\epsilon$ от $10^{-1}$ до $10^{-7}$. Построить график количества вычислений минимизируемой функции от десятичного логарифма задаваемой точности $\epsilon$.
* Реализовать алгоритм поиска интервала, содержащего минимум функций.
* Реализовать метод Фибоначчи, сравнить его с методами дихотомии и золотого сечения.

In [2]:
import numpy as np
import pandas as pd
import math
from IPython.display import display

In [3]:
K1 = (3 - math.sqrt(5)) / 2
K2 = (math.sqrt(5) - 1) / 2
K3 = (1 + math.sqrt(5)) / 2

In [4]:
f = lambda x: (x - 8) ** 2

In [12]:
def FindFib(value):
    a = 1
    b = 1
    c = a + b
    while (c < value):
        v = c
        a = b
        b = v
        c = b + a
    return c, a

### Метод дихотомии

In [39]:
def DichotomyMethod(eps = 10**(-7), a0 = -2.0, b0 = 20.0):
    a, b, x1, x2 = [], [], [], []
    a.append(a0)
    b.append(b0)
    x1.append((a0 + b0 - eps / 2) / 2)
    x2.append((a0 + b0 + eps / 2) / 2)
    n = 1
    while (abs(b[n - 1] - a[n - 1]) > eps):
        if (f(x1[n - 1]) <= f(x2[n - 1])):
            a.append(a[n - 1])
            b.append(x2[n - 1])
        else:
            a.append(x1[n - 1])
            b.append(b[n - 1])
        x1.append((a[n] + b[n] - eps / 2) / 2)
        x2.append((a[n] + b[n] + eps / 2) / 2)
        n += 1
    iArr = range(1, n + 1)
    df = pd.DataFrame({'x1' : x1,
                       'x2' : x2,
                       'f(x1)' : [f(xi) for xi in x1], 
                       'f(x2)' : [f(xi) for xi in x2],
                       'ai' : a,
                       'bi' : b,
                       'li' : [b[i] - a[i] for i in range(n)], 
                       'l(i-1) / li ' : [(b[i] - a[i]) / (b[i] - a[i]) for i in range(n)]},
                        index = iArr)
    display(df)

In [40]:
DichotomyMethod()

,x1,x2,f(x1),f(x2),ai,bi,li,l(i-1) / li
1,9.000000,9.000000,1.000000e+00,1.000000e+00,-2.000000,20.000000,2.200000e+01,1.0
2,3.500000,3.500000,2.025000e+01,2.025000e+01,-2.000000,9.000000,1.100000e+01,1.0
3,6.250000,6.250000,3.062500e+00,3.062500e+00,3.500000,9.000000,5.500000e+00,1.0
4,7.625000,7.625000,1.406250e-01,1.406250e-01,6.250000,9.000000,2.750000e+00,1.0
5,8.312500,8.312500,9.765624e-02,9.765627e-02,7.625000,9.000000,1.375000e+00,1.0
6,7.968750,7.968750,9.765639e-04,9.765608e-04,7.625000,8.312500,6.875000e-01,1.0
7,8.140625,8.140625,1.977538e-02,1.977540e-02,7.968750,8.312500,3.437500e-01,1.0
8,8.054687,8.054688,2.990720e-03,2.990726e-03,7.968750,8.140625,1.718750e-01,1.0
9,8.011719,8.011719,1.373286e-04,1.373297e-04,7.968750,8.054688,8.593755e-02,1.0
10,7.990234,7.990234,9.536788e-05,9.536690e-05,7.968750,8.011719,4.296880e-02,1.0


### Метод золотого сечения

In [37]:
def GoldenRatioMethod(eps = 10 ** (-7), a0 = -2.0, b0 = 20.0):
    x1, x2, xm, a, b = [], [], [], [], []
    a.append(a0)
    b.append(b0)
    x1.append(a0 + K1 * (b0 - a0))
    x2.append(a0 + K2 * (b0 - a0))
    xm.append(0)
    n = 1
    while (abs(b[n - 1] - a[n - 1]) > eps):
        if (f(x1[n - 1]) <= f(x2[n - 1])):
            a.append(a[n - 1])
            b.append(x2[n - 1])
            x2.append(x1[n - 1])
            x1.append(a[n] + b[n] - x1[n - 1])
            xm.append(x1[n - 1])
        else:
            a.append(x1[n - 1])
            b.append(b[n - 1])
            x1.append(x2[n - 1])
            x2.append(a[n] + b[n] - x2[n - 1])
            xm.append(x2[n - 1])
        n += 1
    iArr = range(1, n + 1)
    df = pd.DataFrame({'x1' : x1,
                       'x2' : x2,
                       'f(x1)' : [f(xi) for xi in x1], 
                       'f(x2)' : [f(xi) for xi in x2],
                       'ai' : a,
                       'bi' : b,
                       'li' : [b[i] - a[i] for i in range(n)], 
                       'l(i-1) / li ' : [(b[i] - a[i]) / (b[i] - a[i]) for i in range(n)]},
                        index = iArr)
    display(df)

In [38]:
GoldenRatioMethod()

,x1,x2,f(x1),f(x2),ai,bi,li,l(i-1) / li
1,6.403252,11.596748,2.549603e+00,1.293659e+01,-2.000000,20.000000,2.200000e+01,1.0
2,3.193496,6.403252,2.310249e+01,2.549603e+00,-2.000000,11.596748,1.359675e+01,1.0
3,6.403252,8.386991,2.549603e+00,1.497620e-01,3.193496,11.596748,8.403252e+00,1.0
4,8.386991,9.613009,1.497620e-01,2.601798e+00,6.403252,11.596748,5.193496e+00,1.0
5,7.629270,8.386991,1.374406e-01,1.497620e-01,6.403252,9.613009,3.209757e+00,1.0
6,7.160973,7.629270,7.039663e-01,1.374406e-01,6.403252,8.386991,1.983739e+00,1.0
7,7.629270,7.918694,1.374406e-01,6.610696e-03,7.160973,8.386991,1.226018e+00,1.0
8,7.918694,8.097567,6.610696e-03,9.519402e-03,7.629270,8.386991,7.577208e-01,1.0
9,7.808144,7.918694,3.680879e-02,6.610696e-03,7.629270,8.097567,4.682972e-01,1.0
10,7.918694,7.987017,6.610696e-03,1.685465e-04,7.808144,8.097567,2.894236e-01,1.0


### Метод Фибоначчи

In [43]:
def FibonachiMethod(eps = 10 ** (-7), a0 = -2.0, b0 = 20.0):
    fn2, fn0 = FindFib((b0 - a0) / eps)
    a, b, x1, x2, xm = [], [], [], [], []
    a.append(a0)
    b.append(b0)
    x1.append(a[0] + (b[0] - a[0]) * fn0 / fn2)
    x2.append(a[0] + b[0] - x1[0])
    xm.append(0)
    n = 0
    while (abs(b[n] - a[n]) > eps):
        if (f(x1[n]) <= f(x2[n])):
            a.append(a[n])
            b.append(x2[n])
            x2.append(x1[n])
            x1.append(a[n + 1] + b[n + 1] - x1[n])
            xm.append(x1[n])
        else:
            a.append(x1[n])
            b.append(b[n])
            x1.append(x2[n])
            x2.append(a[n + 1] + b[n + 1] - x2[n])
            xm.append(x2[n])
        n += 1
    iArr = range(1, n + 2)
    df = pd.DataFrame({'x1' : x1,
                       'x2' : x2,
                       'f(x1)' : [f(xi) for xi in x1], 
                       'f(x2)' : [f(xi) for xi in x2],
                       'ai' : a,
                       'bi' : b,
                       'li' : [b[i] - a[i] for i in range(n + 1)], 
                       'l(i-1) / li ' : [(b[i] - a[i]) / (b[i] - a[i]) for i in range(n + 1)]},
                        index = iArr)
    display(df)

In [44]:
FibonachiMethod()

,x1,x2,f(x1),f(x2),ai,bi,li,l(i-1) / li
1,6.403252,11.596748,2.549603e+00,1.293659e+01,-2.000000,20.000000,2.200000e+01,1.0
2,3.193496,6.403252,2.310249e+01,2.549603e+00,-2.000000,11.596748,1.359675e+01,1.0
3,6.403252,8.386991,2.549603e+00,1.497620e-01,3.193496,11.596748,8.403252e+00,1.0
4,8.386991,9.613009,1.497620e-01,2.601798e+00,6.403252,11.596748,5.193496e+00,1.0
5,7.629270,8.386991,1.374406e-01,1.497620e-01,6.403252,9.613009,3.209757e+00,1.0
6,7.160973,7.629270,7.039663e-01,1.374406e-01,6.403252,8.386991,1.983739e+00,1.0
7,7.629270,7.918694,1.374406e-01,6.610696e-03,7.160973,8.386991,1.226018e+00,1.0
8,7.918694,8.097567,6.610696e-03,9.519402e-03,7.629270,8.386991,7.577208e-01,1.0
9,7.808144,7.918694,3.680879e-02,6.610696e-03,7.629270,8.097567,4.682972e-01,1.0
10,7.918694,7.987017,6.610696e-03,1.685465e-04,7.808144,8.097567,2.894236e-01,1.0


### Поиск интервала, содержащего минимум функции

In [45]:
def FindMinimum(a0 = -2.0, b0 = 20.0):
    k = 0
    x = []
    x.append(a0)
    h = 0
    delt = 10 ** (-8)
    if (f(x[0]) > f(x[0] + delt)):
        k += 1
        x.append(x[0] + delt)
        h = delt
    elif (f(x[0]) < f(x[0] + delt)):
        k += 1
        x.append(x[0] - delt)
        h = -delt
    else:
        print ("Govno kakoe-to")
        exit
    
    flag = True
    while (flag):
        h *= 2
        x.append(x[k] + h)
        if (f(x[k]) > f(x[k + 1])):
            k += 1
        else:
            flag = False
            print ('[', x[k - 2], ';', x[k], ']')

In [46]:
FindMinimum()

[ 0.6843545499999999 ; 8.73741823 ]
